# **A full 3D analysis using the low-level Gammapy API**

**Objective: Performing a full 3D anaysis of the extended source [MSH 15-52](http://tevcat.uchicago.edu/?mode=1;id=95)**

In practice, we have to:
- Prepare the **data access and selection**
  - Create a `~gammapy.data.DataStore` poiting to the relevant data 
  - Apply an observation selection to produce a list of observations, a `~gammapy.data.Observations` object.
- Set up the **analyis parameters**
  - Define a geometry of the Map we want to produce, with a sky projection and an energy range.
    - Create a `~gammapy.maps.MapAxis` for the energy
    - Create a `~gammapy.maps.WcsGeom` for the geometry
    - Define the exclusion mask
    - Choose the correct ~gammapy.datasets.Dataset type and define it
- Do the **data reduction**
  - Create the necessary makers : 
    - the map dataset maker : `~gammapy.makers.MapDatasetMaker`
    - the [background normalization](https://docs.gammapy.org/1.1/user-guide/makers/fov.html) maker, here a `~gammapy.makers.FoVBackgroundMaker`
    - and usually the safe range maker : `~gammapy.makers.SafeMaskMaker`
  - Perform the data reduction loop. And for every observation:
    - Apply the makers sequentially to produce the current `~gammapy.datasets.MapDataset`
    - Stack it on the target one.
- Make the **modeling and fitting**
  - Define the `~gammapy.modeling.models.SkyModel` to apply to the dataset.
  - Create a `~gammapy.modeling.Fit` object and run it to fit the model parameters
  - Apply a `~gammapy.estimators.FluxPointsEstimator` to compute flux points for the spectral part of the fit.

As support for this exercice, please refer to the [Low Level API tutorial](https://docs.gammapy.org/1.1/tutorials/starting/analysis_2.html).

## Setup
First, we setup the analysis by performing required imports.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path
import numpy as np
import logging
from astropy import units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion
from scipy.stats import norm

In [ ]:
from gammapy.data import DataStore
from gammapy.datasets import MapDataset
from gammapy.maps import WcsGeom, MapAxis
from gammapy.makers import MapDatasetMaker, SafeMaskMaker, FoVBackgroundMaker
from gammapy.modeling.models import (
    SkyModel,
    PowerLawSpectralModel,
    PointSpatialModel,
    FoVBackgroundModel,
    GaussianSpatialModel,
    Models
)
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator, ExcessMapEstimator

## Optional set-up

In [ ]:
logging.basicConfig()    
log = logging.getLogger("1Danalysis")
log.setLevel(logging.WARNING) #INFO, WARNING, DEBUG

from astropy.io.fits.verify import VerifyWarning
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore', VerifyWarning)

from gammapy.utils import pbar
pbar.SHOW_PROGRESS_BAR = True

## Defining the datastore and selecting observations

We first use the `~gammapy.data.DataStore` object to access the observations we want to analyse, here the H.E.S.S. DL3 DR1. 

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")

In [ ]:
pos = SkyCoord(228.32083333, -59.08166667, unit=u.deg, frame="icrs")
pos

We can now define an observation filter to select only the relevant observations. 
Here we use a cone search which we define with a python dict.

We then filter the `ObservationTable` with `~gammapy.data.ObservationTable.select_observations()`.

In [ ]:
obs_table_filtered = data_store.obs_table.select_sky_circle(center=pos, radius=2 * u.deg)
obs_ids = obs_table_filtered["OBS_ID"]

We can now retrieve the relevant observations by passing their `obs_id` to the`~gammapy.data.DataStore.get_observations()` method.

In [ ]:
observations = data_store.get_observations(obs_ids)

In [ ]:
# print(observations)

In [ ]:
obs = observations[0]

In [ ]:
obs.events.peek()

## Preparing reduced datasets geometry

Now we define a reference geometry for our analysis, We choose a WCS based geometry with a binsize of 0.02 deg and also define an energy axis: 

In [ ]:
energy_axis = MapAxis.from_energy_bounds(0.3, 20.0, 20, unit="TeV")

geom = WcsGeom.create(
    skydir=(pos.ra.value, pos.dec.value),
    binsz=0.02,
    width=(4, 4),
    frame="icrs",
    proj="CAR",
    axes=[energy_axis],
)

# Reduced IRFs are defined in true energy (i.e. not measured energy).
energy_axis_true = MapAxis.from_energy_bounds(
    0.1, 40, 40, unit="TeV", name="energy_true"
)

Now we can define the target dataset with this geometry.

In [ ]:
stacked = MapDataset.create(
    geom=geom, energy_axis_true=energy_axis_true, name="msh-stacked"
)

## Data reduction

### Create the maker classes to be used

We first initialize the `Maker` objects that will take care of the data reduction.

In [ ]:
offset_max = 2.0 * u.deg
maker = MapDatasetMaker()
maker_safe_mask = SafeMaskMaker(
    methods=["offset-max", "aeff-max", "bkg-peak"], offset_max=offset_max
)

In [ ]:
circle = CircleSkyRegion(
    center=SkyCoord(pos.ra, pos.dec), radius=0.4 * u.deg
)
exclusion_mask = geom.region_mask(regions=[circle], inside=False)
maker_fov = FoVBackgroundMaker(method="scale", exclusion_mask=exclusion_mask)

In [ ]:
exclusion_mask.plot_interactive()

### Perform the data reduction loop

In [ ]:
%%time

for obs in observations:
    # First a cutout of the target map is produced
    cutout = stacked.cutout(
        obs.get_pointing_icrs(obs.tmid), width=2 * offset_max, name=f"obs-{obs.obs_id}"
    )
    # A MapDataset is filled in this cutout geometry
    dataset = maker.run(cutout, obs)
    # The data quality cut is applied
    dataset = maker_safe_mask.run(dataset, obs)
    # fit background model
    dataset = maker_fov.run(dataset)
    print(
        f"Background norm obs {obs.obs_id}: "
        f"{dataset.background_model.spectral_model.norm.value:.2f} +/- {dataset.background_model.spectral_model.norm.error:.3f}"
    )
    # The resulting dataset cutout is stacked onto the final one
    stacked.stack(dataset)

In [ ]:
print(stacked)

### Inspect the reduced dataset

In [ ]:
stacked.counts.smooth(0.1 * u.deg).plot_interactive(
    stretch="linear", add_cbar=True
);

In [ ]:
dataset.mask_safe.plot_grid(vmin=0, vmax=1);

Save the dataset to disc using `~gammapy.datasets.MapDataset.write()` method:

In [ ]:
# filename = "msh-stacked-dataset.fits.gz"
# stacked.write(filename, overwrite=True)

In [ ]:
region = CircleSkyRegion(
    center=SkyCoord(228.5, -59.15, unit="deg"), radius=0.4 * u.deg
)
stacked.plot_residuals(kwargs_spectral={"region":region});

At this stage, we have created a DL4 `MapDataset`. The rest of the notebook is agnostic to how this dataset was created (simulated/H.E.S.S./Fermi-LAT, etc)...

If you are interested only in the spectrum, not the morphology, you can do a 1D spectral extraction using the `SpectrumDatasetMaker`. See https://docs.gammapy.org/1.1/tutorials/index.html#d-spectral for details.

## Compute an excess and a significance map

In [ ]:
estimator = ExcessMapEstimator(0.04 * u.deg, selection_optional=[])
lima_maps = estimator.run(stacked)

In [ ]:
significance_map = lima_maps["sqrt_ts"]
excess_map = lima_maps["npred_excess"]

In [ ]:
# We can plot the excess and significance maps
plt.figure(figsize=(10, 10))
ax1 = plt.subplot(221, projection=significance_map.geom.wcs)
ax2 = plt.subplot(222, projection=excess_map.geom.wcs)
plt.subplots_adjust(right=1.0)

ax1.set_title("Significance map")
significance_map.plot(ax=ax1, add_cbar=True);

ax2.set_title("Excess map")
excess_map.plot(ax=ax2, add_cbar=True);

In [ ]:
## You can zoom into your region
significance_map.cutout(position=SkyCoord(228.5, -59.15, unit="deg"), width=0.8*u.deg).plot(add_cbar=True);

## Define the model
We first define the model, a `SkyModel`, as the combination of a point source `SpatialModel` with a powerlaw `SpectralModel`:

In [ ]:
spatial_model = GaussianSpatialModel(
    lon_0=pos.ra, lat_0=pos.dec, sigma=0.3*u.deg, frame="icrs"
)
# We limit the model position inside a 1 deg box centered on the reference position of MSH 15-52
spatial_model.lon_0.min = spatial_model.lon_0.value - 0.5
spatial_model.lon_0.max = spatial_model.lon_0.value + 0.5
spatial_model.lat_0.min = spatial_model.lat_0.value - 0.5
spatial_model.lat_0.max = spatial_model.lat_0.value + 0.5

spectral_model = PowerLawSpectralModel(
    index=2.702,
    amplitude=4.712e-11 * u.Unit("1 / (cm2 s TeV)"),
    reference=1 * u.TeV,
)

sky_model = SkyModel(
    spatial_model=spatial_model, spectral_model=spectral_model, name="msh"
)

Now, we define a global `~gammapy.modeling.model.FoVBackgroundModel` in order to finely adjust the level of residual CR backgroud. This should **not** be forgotten.

In [ ]:
bkg_model = FoVBackgroundModel(dataset_name="msh-stacked")

Now we assign these models to our reduced dataset:

In [ ]:
stacked.models = Models([sky_model, bkg_model])

In [ ]:
print(stacked)

## Fit the model

The `~gammapy.modeling.Fit` class is orchestrating the fit, connecting the `stats` method of the dataset to the minimizer. By default, it uses `iminuit`.

Its constructor takes a list of dataset as argument.

In [ ]:
mask_energy = stacked.counts.geom.energy_mask(650 * u.GeV, 20 * u.TeV)
stacked.mask_fit = mask_energy

In [ ]:
%%time
fit = Fit(optimize_opts={"print_level": 1})
result = fit.run([stacked])

The `FitResult` contains information about the optimization and parameter error calculation.

In [ ]:
result.success

The fitted parameters are visible from the `~astropy.modeling.models.Models` object.

In [ ]:
fitted_param = stacked.models.to_parameters_table()
# fitted_param.pprint_all()
fitted_param.show_in_notebook()

### Inspecting residuals

For any fit it is useful to inspect the residual images. We have a few options on the dataset object to handle this. First we can use `.plot_residuals_spatial()` to plot a residual image, summed over all energies:

In [ ]:
region = CircleSkyRegion(
    center=pos, radius=0.4 * u.deg
)
stacked.plot_residuals(kwargs_spectral={"region":region});

## Plot the fitted spectrum

### Making a butterfly plot 

The `SpectralModel` component can be used to produce a, so-called, butterfly plot showing the envelope of the model taking into account parameter uncertainties:

In [ ]:
spec = sky_model.spectral_model

Now we can actually do the plot using the `plot_error` method:

In [ ]:
energy_bounds = [1, 10] * u.TeV
spec.plot(energy_bounds=energy_bounds, sed_type="e2dnde")
ax = spec.plot_error(energy_bounds=energy_bounds, sed_type="e2dnde")

### Computing flux points

We can now compute some flux points using the `~gammapy.estimators.FluxPointsEstimator`. 

Besides the list of datasets to use, we must provide it the energy intervals on which to compute flux points as well as the model component name. 

In [ ]:
energy_edges = [0.5, 1, 2, 4, 10, 20] * u.TeV
fpe = FluxPointsEstimator(energy_edges=energy_edges, source="msh", selection_optional="all", n_sigma_ul=3)

In [ ]:
%%time
flux_points = fpe.run(datasets=[stacked])

In [ ]:
flux_points.to_table(sed_type="dnde", formatted=True)

In [ ]:
ax = spec.plot_error(energy_bounds=[0.3, 50] * u.TeV, sed_type="e2dnde")
ax.set_xlim(0.2, 70)
ax.set_ylim(1.e-12, 4.e-11)
flux_points.plot(ax=ax, sed_type="e2dnde");

# Exercises

## Beginner 
- Since MSH 1552 is a Galactic source, it can make sense to analyze it using Galactic coordinates instead of ICRS. Try to repeat the data reduction in the Galactic coordinate frame
- Modify the data reduction loop to fit also the background model `tilt` parameter, in addition to the default `norm` fit
- By default the model fit is performed in the full energy range defined by the dataset `mask_safe`. Try to restrict it by using the `Dataset.mask_fit` property
- Repeat the flux points estimation by reoptimizing, in each energy bin, all the model free parameters. This will take longer, but it will provide a more reliable estimation of the source flux in each energy bin
- What is the TS-based significance of MSH 1552? You can estimate it by comparing the model likelihood without (null hypothesis) and with the source model 

## Advanced 
- Is the source extended? You can try to understand this (without any model fitting) by producing a radial flux profiles using annulus regions concentric around the source  (Tutorial reference: https://docs.gammapy.org/1.1/tutorials/analysis-3d/flux_profiles.html).
- Has the source an energy-dependant morphology? A dedicated estimator has been implemented to answer to this question, `EnergyDependentMorphologyEstimator`
- Try to estimate the level of analysis systematics related to our imperfect knowledge of the IRFs. For example, what is the impact of a spectral bias in the background model with respect to the data? (Hint: A simulation-based approach can help you tackle this problem!)